# Setting up Pôle emploie API

## Importing useful tools, librairies..

In [1]:
pip install api-offres-emploi

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from tqdm.notebook import trange, tqdm
import time
from datasets import load_dataset
import sys, os

In [3]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

**Using private key to acess the api. To create your own keys : https://pole-emploi.io/inscription.**

In [4]:
with open('cles_api.txt') as f:
    lines = f.readlines()
    cles_api = { "client_id" : lines[0][:-1], "client_secret" : lines[1]}


In [5]:
cles_api

{'client_id': 'PAR_classifieurcoderome_e939f0b73f15ab1364fd65c4e2e063a35a8c904ba0d242df6e0606928ce8cde9',
 'client_secret': '3e3582e7cd04a4bc733f9f3a984e701f2b13d3f8e023e7c172eca0fa0a887de3'}

In [6]:
from offres_emploi import Api
api = Api(client_id= cles_api['client_id'], 
             client_secret= cles_api['client_secret'])

## Getting started with the api

In [7]:
test_Chauffeur  = api.search(params = {'codeROME' : 'A1101'})

filters = test_Chauffeur['filtresPossibles']
results =  test_Chauffeur['resultats']
content_range = test_Chauffeur['Content-Range']
n_offers = content_range['max_results']

results_df = pd.DataFrame(results)

results_df.head(2)

Making request with params {'codeROME': 'A1101'}
Token has not been requested yet. Requesting token
Now requesting token


,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,qualificationLibelle,secteurActivite,secteurActiviteLibelle,origineOffre,qualitesProfessionnelles,offresManqueCandidats,experienceCommentaire,permis,complementExercice,formations
0,130YVCS,Tractoriste agricole,"Au sein d'une exploitation agricole, vous effe...",2022-04-01T16:28:45.000Z,2022-04-01T16:28:45.000Z,"{'libelle': '30 - GENERAC', 'latitude': 43.735...",A1101,Conduite d'engins agricoles et forestiers,Tractoriste agricole,{'entrepriseAdaptee': False},...,"Ouvrier qualifié (P1,P2)",01,Culture de la vigne,"{'origine': '1', 'urlOrigine': 'https://candid...",NaN,NaN,NaN,NaN,NaN,NaN
1,130YKZD,Tractoriste agricole (H/F),La SCEA Domaine de Cransac recrute 1 Tractoris...,2022-04-01T14:33:38.000Z,2022-04-01T14:35:44.000Z,"{'libelle': '31 - FRONTON', 'latitude': 43.854...",A1101,Conduite d'engins agricoles et forestiers,Tractoriste agricole,"{'nom': ' DOMAINE DE CRANSAC', 'entrepriseAdap...",...,"Ouvrier qualifié (P1,P2)",01,Culture de la vigne,"{'origine': '1', 'urlOrigine': 'https://candid...","[{'libelle': 'Capacité d'adaptation', 'descrip...",False,NaN,NaN,NaN,NaN


In [8]:
n_offers

'850'

In [9]:
results_df[['description', "romeCode"]]

,description,romeCode
0,"Au sein d'une exploitation agricole, vous effe...",A1101
1,La SCEA Domaine de Cransac recrute 1 Tractoris...,A1101
2,Nous cherchons un tractoriste confirmé pour di...,A1101
3,Vous réalisez des travaux de chargement épande...,A1101
4,Poste en Juillet\n\nVous avez une expérience d...,A1101
...,...,...
145,"Au sein d'une entreprise agroalimentaire, vous...",A1101
146,L'agence CRIT DOUAI recherche un Agent d'entre...,A1101
147,Dans le cadre de l'alternance et après une for...,A1101
148,Aladin.farm by InVivo est une plateforme digit...,A1101


In [10]:
results_df['description'][4]

"Poste en Juillet\n\nVous avez une expérience de la conduite de tracteur avec remorque pour le transport de la récolte d'abricots des champs à l'exploitation.\n\nVous serez en appui aux équipes lors des récoltes.\n\nLe salaire est à débattre en fonction de vos qualifications et de votre expérience. \n\nLe poste est non logé, possibilité de camping sur place avec des sanitaires et une cuisine à disposition. Animaux non admis. \n "

## Creating subcategories for jobs according to the codeRome classification

In [11]:
dataset_train = load_dataset("oscarfossey/NLP_Pole_emploi", data_files='ROME_ArboPrincipale.csv')

Using custom data configuration oscarfossey--NLP_Pole_emploi-7fcd601cddff1243
Reusing dataset csv (C:\Users\admin\.cache\huggingface\datasets\csv\oscarfossey--NLP_Pole_emploi-7fcd601cddff1243\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
df_main =  pd.DataFrame(dataset_train['train'])

In [13]:
df_main.head()

,,.1,.2,.3,Code OGR
0,A,,,"Agriculture et Pêche, Espaces naturels et Espa...",
1,A,11,,Engins agricoles et forestiers,
2,A,11,01,Conduite d'engins agricoles et forestiers,
3,A,11,01,Chauffeur / Chauffeuse de machines agricoles,11987
4,A,11,01,Conducteur / Conductrice d'abatteuses,12862


**Renaming columns**

In [14]:
df_main.rename(columns = {' ' : "categorie" , ' .1' : "sub_categorie", ' .2' : "sub_sub_categorie", ' .3' : "name" }, inplace=True)

df_main["code_rome"] = df_main["categorie"] + df_main["sub_categorie"] + df_main["sub_sub_categorie"]

**Keeping the categorie name in a dataset**

In [15]:
df_categorie_name = df_main.loc[df_main["sub_categorie"] ==  ' ']
df_sub_categorie_name = df_main.loc[(df_main["sub_categorie"] !=  ' ') & (df_main["sub_sub_categorie"] ==  ' ')]
df_sub_sub_categorie_name = df_main.loc[
                                        (df_main["sub_categorie"] !=  ' ') 
                                        & (df_main["sub_sub_categorie"] !=  ' ')
                                        & (df_main["Code OGR"] ==  ' ')]

df_categorie_name.drop(["Code OGR"], axis = 1, inplace = True)
df_sub_categorie_name.drop(["Code OGR"], axis = 1, inplace = True)
df_sub_sub_categorie_name.drop(["Code OGR"], axis = 1, inplace = True)

D:\Logiciel\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


**Displaying the name of the 14 main categories.**

In [16]:
print("Number of categorie : ", df_categorie_name.count()[0])
print("Number of sub_categorie : ", df_sub_categorie_name.count()[0])
print("Number of sub_sub_categorie : ", df_sub_sub_categorie_name.count()[0])

df_categorie_name

Number of categorie :  14
Number of sub_categorie :  110
Number of sub_sub_categorie :  532


,categorie,sub_categorie,sub_sub_categorie,name,code_rome
0,A,,,"Agriculture et Pêche, Espaces naturels et Espa...",A
626,B,,,Arts et Façonnage d'ouvrages d'art,B
1001,C,,,"Banque, Assurance, Immobilier",C
1396,D,,,"Commerce, Vente et Grande distribution",D
2108,E,,,"Communication, Média et Multimédia",E
2738,F,,,"Construction, Bâtiment et Travaux publics",F
3571,G,,,"Hôtellerie-Restauration, Tourisme, Loisirs et ...",G
4156,H,,,Industrie,H
6954,I,,,Installation et Maintenance,I
7699,J,,,Santé,J


## Creation of the datasets

In [17]:
columns = ["textual description", "code_rome"]
df_text  = pd.DataFrame(columns = columns)

df_text.head()

,textual description,code_rome


**Dowloadind the data straight from the api**

*We will display an error message when a sub_sub_categorie do not have any job offers associated on pôle emploi.*

In [18]:
error_count = 0
for code_rome in tqdm(df_sub_sub_categorie_name.code_rome):
    try:
        with HiddenPrints():
            request  = api.search(params = {'codeROME' : code_rome});
        if len(request['resultats']) > 1:
            df = pd.DataFrame(request['resultats'])[['description', "romeCode"]]
            df.rename(columns = {'description' : "textual description",  'romeCode' : "code_rome" }, inplace=True)
            df_text = pd.concat([df_text, df])
    except:
        print("Error number for missing Job description: ", error_count + 1)
        error_count +=1
    time.sleep(0.4)

  0%|          | 0/532 [00:00<?, ?it/s]

Error number for missing Job description:  1
Error number for missing Job description:  2
Error number for missing Job description:  3
Error number for missing Job description:  4


In [19]:
df_text.head(3)

,textual description,code_rome
0,"Au sein d'une exploitation agricole, vous effe...",A1101
1,La SCEA Domaine de Cransac recrute 1 Tractoris...,A1101
2,Nous cherchons un tractoriste confirmé pour di...,A1101


**Merging categories and textual description**

In [20]:
dataset = df_text.merge(df_sub_sub_categorie_name, how='left', on='code_rome')

dataset.head(3)

,textual description,code_rome,categorie,sub_categorie,sub_sub_categorie,name
0,"Au sein d'une exploitation agricole, vous effe...",A1101,A,11,01,Conduite d'engins agricoles et forestiers
1,La SCEA Domaine de Cransac recrute 1 Tractoris...,A1101,A,11,01,Conduite d'engins agricoles et forestiers
2,Nous cherchons un tractoriste confirmé pour di...,A1101,A,11,01,Conduite d'engins agricoles et forestiers


**Deleting empty description**

In [21]:
dataset.count()

textual description    64230
code_rome              64239
categorie              64239
sub_categorie          64239
sub_sub_categorie      64239
name                   64239
dtype: int64

In [22]:
dataset.dropna(subset=['textual description'], inplace = True)

dataset.count()

textual description    64230
code_rome              64230
categorie              64230
sub_categorie          64230
sub_sub_categorie      64230
name                   64230
dtype: int64

**Saving useful datasets in a local folder (optionnal because disponible at https://huggingface.co/datasets/oscarfossey/NLP_Pole_emploi)**

In [23]:
#dataset.to_csv('Textual_description.csv',index=False)

In [24]:
#df_categorie_name.to_csv('tags_name.csv',index=False)
#df_sub_categorie_name.to_csv('sub_tags_name.csv',index=False)
#df_sub_sub_categorie_name.to_csv('sub_sub_tags_name.csv',index=False)